# **Overview **
Will be answering the following questions utilizing Bigquery SQL Statements <br>
- What is the total number of transactions generated per device browser in July 2017? <br>
- The real bounce rate is defined as the percentage of visits with a single pageview. What was the real bounce rate per traffic source? <br>
- What was the average number of product pageviews for users who made a purchase in July 2017? <br>
- What was the average number of product pageviews for users who did not make a purchase in July 2017? <br>
- What was the average total transactions per user that made a purchase in July 2017? <br>
- What is the average amount of money spent per session in July 2017?

# **Context** <br>
The Google Merchandise Store sells Google branded merchandise. The data is typical of what you would see for an ecommerce website.

# **Content** <br>
The sample dataset contains Google Analytics 360 data from the Google Merchandise Store, a real ecommerce store. The Google Merchandise Store sells Google branded merchandise.It includes the following kinds of information:

Traffic source data: information about where website visitors originate. This includes data about organic traffic, paid search traffic, display traffic, etc. Content data: information about the behavior of users on the site. This includes the URLs of pages that visitors look at, how they interact with content, etc. Transactional data: information about the transactions that occur on the Google Merchandise Store website.

# **Notes:** <br>
Will be using the official Google BigQuery Syntax, but another library exists on Kaggle called BigQueryHelper that simplifies read-only tasks in BigQuery and outputs data into pandas dataframes. 
https://www.kaggle.com/sohier/introduction-to-the-bq-helper-package <br>
Would have to manually convert queries to dataframes otherwise.

Google's Documentation for BigQuery:
https://googleapis.github.io/google-cloud-python/latest/bigquery/index.html

# **Reading BigQuery Data**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline  


from google.cloud import bigquery
import pandas as pd

In [ ]:
# initiate bigquery client
int_query = bigquery.Client()

In [ ]:
query1 = """
SELECT *
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*` ga
WHERE _TABLE_SUFFIX BETWEEN '20170701' AND '20170731'
limit 10
        """

data = int_query.query(query1).to_dataframe()
data.head()

# **Data Observations** <br>
* The data is using JSON formats for the columns totals, trafficSource,device,geoNetwork,customDimensions, and hits.
* visitNumber refers to the nth visit somene is visiting the site (i.e visitNumber 7 means it is the seventh time the same person has visited)
* There seems to be a difference between visitorId and fullVisitorID, userId and clientId
* The date format is in %Y%m%d and could be converted to a datetime in different format for other analyses 


**Expanding JSON fields in the geonetwork Column** <br>
The purpose of expanding this field is to see what other fields exist due to the JSON format which stores more fields within the columns

In [ ]:
query1 = """
SELECT
geonetwork
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`
WHERE _TABLE_SUFFIX BETWEEN '20170701' AND '20170731'
limit 10

        """

data = int_query.query(query1).to_dataframe()

#Will expand width of the column to see entire value of the field
pd.set_option('display.max_colwidth',-1)
data.head()

The column geonetwork includes the following fields which can be specified in the queries:
* continent
* subContinent
* country
* region
* metro
* city
* cityid
* networkDomain
* latitude
* longitude
* networkLocation

**Retreiving list of countries**

In [ ]:
query1 = """
SELECT
distinct geonetwork.country
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`
WHERE _TABLE_SUFFIX BETWEEN '20170701' AND '20170731'
order by 1 desc

        """

data = int_query.query(query1).to_dataframe()
pd.set_option('display.max_colwidth',-1)
data.head(15)

**Expanding JSON fields in the totals Column**

In [ ]:
query1 = """
SELECT
totals
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`
WHERE _TABLE_SUFFIX BETWEEN '20170701' AND '20170731'
limit 10

        """

data = int_query.query(query1).to_dataframe()

#Will expand width of the column to see entire value of the field
pd.set_option('display.max_colwidth',-1)
data.head(10)

The column geonetwork includes the following fields which can be specified in the queries:
* visits
* hits
* pageviews
* timeOnSite
* bounces
* transactions (refers to the number of transactions)
* transactionRevenue (refers to the dollar amount of transaction)
* newVisits
* screenviews
* uniqueScreenviews
* timeOnScreen
* totalTransactionRevenue
* sessionQualityDim

**Expanding JSON fields in the trafficSource Column**

In [ ]:
query1 = """
SELECT
trafficSource
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`
WHERE _TABLE_SUFFIX BETWEEN '20170701' AND '20170731'
limit 10

        """

data = int_query.query(query1).to_dataframe()

#Will expand width of the column to see entire value of the field
pd.set_option('display.max_colwidth',-1)
data.head(3)

In [ ]:
#Resetting default options

#resets the display options which were modified earlier to view JSON fields
pd.reset_option('^display.')

#resets every option
#pd.reset_option('all')

# **Data Analysis (Questions and Answers)** <br>
*For the purposes of this excerise I will be fitlering out the data by United States only*

### **What is the total number of transactions generated per device browser in July 2017?**

In [ ]:
transactions = """
SELECT
device.browser,
geonetwork.country,
sum(totals.transactions) as total_transactions_value
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`
WHERE _TABLE_SUFFIX BETWEEN '20170701' AND '20170731'
AND geonetwork.country = 'United States'
group by 1,2
order by 3 desc

        """

data = int_query.query(transactions).to_dataframe()
data.head(10)


### **The real bounce rate is defined as the percentage of visits with a single pageview. What was the real bounce rate per traffic source?**

In [ ]:
bounce_rate = """
SELECT
trafficSource.source,
sum(case when totals.pageviews = 1 then totals.visits else 0 end) as bounce_visits,
sum(totals.visits) as total_visits,
sum(case when totals.pageviews = 1 then totals.visits else 0 end) / sum(totals.visits) as bounce_rate
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`
WHERE _TABLE_SUFFIX BETWEEN '20170701' AND '20170731'
AND geonetwork.country = 'United States'
group by 1
order by 3 desc

        """

data = int_query.query(bounce_rate).to_dataframe()
data.head()

### **What was the average number of product pageviews for users who made a purchase in July 2017?**

In [ ]:
pageview_transactions = """
SELECT
distinct totals.transactions,
avg(totals.pageviews) as avg_pageviews
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`
WHERE _TABLE_SUFFIX BETWEEN '20170701' AND '20170731'
AND geonetwork.country = 'United States'
AND totals.transactionRevenue is not null
group by 1
order by 1 asc


        """

data = int_query.query(pageview_transactions).to_dataframe()
data.head()

### **What was the average number of product pageviews for users who did not make a purchase in July 2017?**

In [ ]:
pageview_notransactions = """
SELECT
distinct totals.transactions,
avg(totals.pageviews) as avg_pageviews
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`
WHERE _TABLE_SUFFIX BETWEEN '20170701' AND '20170731'
AND geonetwork.country = 'United States'
AND totals.transactionRevenue is null
group by 1
order by 1 asc


        """

data = int_query.query(pageview_notransactions).to_dataframe()
data.head()

### **What was the average total transactions per user that made a purchase in July 2017?**

In [ ]:
avg_transaction = """
SELECT
avg(totals.transactions) as avg_transaction
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`
WHERE _TABLE_SUFFIX BETWEEN '20170701' AND '20170731'
AND geonetwork.country = 'United States'
AND totals.transactionRevenue is not null
limit 100
        """

data = int_query.query(avg_transaction).to_dataframe()
data


### **What is the average amount of money spent per session in July 2017?**

In [ ]:
avg_transaction = """
SELECT
distinct visitNumber,
avg(totals.totalTransactionRevenue) as avg_transaction
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`
WHERE _TABLE_SUFFIX BETWEEN '20170701' AND '20170731'
AND geonetwork.country = 'United States'
#AND totals.totalTransactionRevenue is not null
group by 1
order by 2 desc
limit 100
        """

data = int_query.query(avg_transaction).to_dataframe()
data.head()

# **Summary** <br>
* Most customers are buying through the Chrome browser with a total of 948 transactions in the month of July. <br>
* The bounce rate is over 30% through the most visited sources <br>
    (bounce rate is defined as the percentage of people who visit the site and leave after viewing the first page) <br>
* The average number of pageviews a customer goes through before making a purchase ranges from an average of 25-55 pageviews <br>
* The average number of pageviews a non-paying customer goes through is approximately 4 pageviews <br>
* The average number of transactions in the month of July was 1 <br>

Most customers that make a transaction do so through the Chrome browser which may be due to the type of customers, web design flaws related to the browser, or marketing related advertising. The bounce rate is relatively high at over 30% through the most trafficed sources, which may indicate a need for a better UX design on the landing page. There is a large discrepancy in pageviews between paying and non-paying customers, which may be an indicator of determing who is likely to make a purchase. This may enable strong targed promotions based on website activity.

### **Possible Further Analysis**
The data can broken down further by understanding returning visitors vs new visitors. New visitors are indicators of growth, while analysis on returning vistitors would help understand user behavior on the site. The data can also be broken out by countries in order to better understand purchasing behavior on a country level. 


